In [2]:
import os
import itertools
from dataclasses import dataclass, field
from code.train import train
from code.optimizers import Optimizer
# from code.problems import Problem
from code.problem import Loss
from code.datasets import Dataset
from code.models import Model

# %matplotlib widget
%load_ext autoreload
%autoreload 1

In [3]:
def zip_dict(**kwargs):
    keys = kwargs.keys()
    for instance in zip(*kwargs.values()):
        yield dict(zip(keys, instance))


def product_dict(**kwargs):
    keys = kwargs.keys()
    for instance in itertools.product(*kwargs.values()):
        yield dict(zip(keys, instance))

In [4]:
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
os.environ["MKL_THREADING_LAYER"] = "AMD"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_DEVICE"] = "cuda:2"

In [5]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_CHECK_EXIST'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

# Mean Estimation

In [ ]:
os.environ["CUDA_DEVICE"] = "cpu"
@dataclass
class BaseConfig():
    n_iters: int = 5000
    n_peers: int = 150
    seed: int = 2

    model: Model = field(default_factory=lambda: Model.Mean)
    loss: Loss = field(default_factory=lambda: Loss.MSE)
    dataset: Dataset = field(default_factory=lambda: Dataset.Normal)

    n_samples: int = 1000
    mu_normal: float = None

    optimizer: Optimizer = field(default_factory=lambda: Optimizer.SGD)
    batch_size: int = 100
    lr: float = 1e-2

    true_weights: bool = None

    md_n_iters_: int = None
    md_full_: bool = None
    md_lr_: int = None

In [ ]:
parameters = {
    'mu_normal': [0.001, 0.01, 0.1],
    'md_lr_': [0.2, 0.9, 3.5]
    # 'seed': [0],
    }

parameters = {
    'mu_normal': [0.001],
    'md_lr_': [0.5],
    # 'seed': [0],
    }

In [ ]:
for cfg in product_dict(**parameters):

    os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
    config = BaseConfig(**cfg)
    config.optimizer = Optimizer.MeritFed
    config.md_full_ = True
    config.md_n_iters_ = 20
    %time train(config)

#     os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
#     config = BaseConfig(**cfg)
#     config.optimizer = Optimizer.MeritFed
#     config.md_full_ = False
#     config.md_n_iters_ = 20
#     %time train(config)

    cfg['md_lr_'] = None

#     os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
#     config = BaseConfig(**cfg)
#     config.optimizer = Optimizer.SGD
#     config.true_weights = False
#     %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
    config = BaseConfig(**cfg)
    config.optimizer = Optimizer.SGD
    config.true_weights = True
    %time train(config)

# 4 classes MNIST

In [6]:
@dataclass
class BaseConfig():
    n_iters: int = 6000
    n_peers: int = 20
    seed: int = 0

    model: Model = field(default_factory=lambda: Model.MNISTNet)
    loss: Loss = field(default_factory=lambda: Loss.CrossEntropy)
    dataset: Dataset = field(default_factory=lambda: Dataset.MNIST)

    n_samples: int = 1000
    h_ratio: float = None

    optimizer: Optimizer = field(default_factory=lambda: Optimizer.SGD)
    batch_size: int = 200
    lr: float = 1e-2

    true_weights: bool = None

    md_n_iters_: int = None
    md_full_: bool = None
    md_lr_: int = None

In [7]:
parameters = {
    'h_ratio': [0.99],
    # 'h_ratio': [0.99, 0.8, 0.9, 0.7, 0.5, 0.3, 0.1, 0.01, 0.05, 0.02],
    # 'h_ratio': [0.7, 0.5],
    'md_lr_': [0.9],
    # 'seed': [0],
    }

In [8]:
for cfg in product_dict(**parameters):

    os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
    config = BaseConfig(**cfg)
    config.optimizer = Optimizer.MeritFed
    config.md_full_ = True
    config.md_n_iters_ = 21
    # config.md_lr_ = 0.05
    %time train(config)

#     os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
#     config = BaseConfig(**cfg)
#     config.optimizer = Optimizer.MeritFed
#     config.md_full_ = False
#     config.md_n_iters_ = 20
#     # config.md_lr_ = 0.05
#     %time train(config)

#     cfg['md_lr_'] = None

#     os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
#     config = BaseConfig(**cfg)
#     config.optimizer = Optimizer.SGD
#     config.true_weights = False
#     %time train(config)

#     cfg['h_ratio'] = None

#     os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
#     config = BaseConfig(**cfg)
#     config.optimizer = Optimizer.SGD
#     config.true_weights = True
#     %time train(config)

100%|██████████| 9912422/9912422 [00:00<00:00, 20116672.89it/s]


Extracting /tmp/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 4602769.73it/s]


Extracting /tmp/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5644128.21it/s]


Extracting /tmp/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 21550371.91it/s]


Extracting /tmp/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/MNIST/raw



KeyboardInterrupt: 